In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd

In [2]:
def scroll_into_view(element):
    navegador.execute_script("arguments[0].scrollIntoView(true);", element)
    return

In [3]:
caminho = f'C:/Users/cayog/Documents/Python/Automação/dep.01-Parceiros de negócios.xlsx'
tabela = pd.read_excel(caminho)
display(tabela)

,DEPÓSITO,Parceiro de Negócio,Unnamed: 2,ESPÉCIE,UNIDADE,Valor
0,Depósito 11,ADAO DIAS TEIXEIRA,ADAO DIAS TEIXEIRA,Truck,Metro Cúbico,13.55
1,Depósito 11,ADILSON DA SILVA OLIVEIRA,adilson da silva oliveira,Truck,Metro Cúbico,12.55
2,Depósito 11,ADRIANA MICHELON,ADRIANA MICHELON,Truck,Metro Cúbico,12.55
3,Depósito 11,ADRIANO DOS SANTOS,adriano dos santos,Truck,Metro Cúbico,12.55
4,Depósito 11,AFONSO FRANCA CONSTRUCOES E COMERCIO LTDA,AFONSO FRANCA CONSTRUCOES E COMERCIO LTDA,Truck,Metro Cúbico,20.55
...,...,...,...,...,...,...
253,Depósito 11,WILLIAM DA CRUZ,william da cruz,Truck,Metro Cúbico,12.55
254,Depósito 11,WILSON MIYASATO,WILSON MIYASATO,Truck,Metro Cúbico,14.55
255,Depósito 11,WILSON ROBERTO FERNANDES,wilson roberto fernandes,Truck,Metro Cúbico,12.55
256,Depósito 11,WLAMIR MOONS ESCABIO JUNIOR,WLAMIR MOONS ESCABIO JUNIOR,Truck,Metro Cúbico,12.55


In [4]:
navegador = webdriver.Chrome()
navegador.get('https://homologa.sincsuite.com.br')

WebDriverWait(navegador, 5).until(EC.presence_of_element_located((By.ID, 'txtUser')))

login = navegador.find_element(By.ID, 'txtUser')
login.send_keys('cayo.garcia' + Keys.TAB)

senha = navegador.find_element(By.ID, 'txtPwd')
senha.send_keys('Flavia01@#')

navegador.find_element(By.XPATH, '//*[@id="btnLogin"]').click()

WebDriverWait(navegador, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="visionTitle54"]')))

navegador.find_element(By.XPATH, '//*[@id="visionTitle54"]').click()
time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="panelTitle58"]').click()

iframe = navegador.find_element(By.XPATH, '//*[@id="frmPrincipal"]')
navegador.switch_to.frame(iframe)

for index, linha in tabela.iterrows():
    WebDriverWait(navegador, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div/table/tbody/tr/td[1]/div/a[1]')))
    
    try:
        elemento = navegador.find_element(By.XPATH, '/html/body/div[3]/div/table/tbody/tr/td[1]/div/a[1]')
        actions = ActionChains(navegador)
        actions.move_to_element(elemento).click().perform()
    except ElementClickInterceptedException:
        WebDriverWait(navegador, 5).until_not(EC.visibility_of_element_located((By.XPATH, '//*[@id="modalPanel"]')))
        actions.move_to_element(elemento).click().perform()

    time.sleep(3)
    navegador.find_element(By.XPATH, '//*[@id="modalPanel_modalDialog"]/div/div[2]/table[1]/tbody/tr[1]/td[3]/div/button').click()
    selecao = navegador.find_element(By.XPATH, '//*[@id="modalPanel_modalDialog"]/div/div[2]/table[1]/tbody/tr[1]/td[3]/div/div/div[1]/input')
    selecao.send_keys(linha["DEPÓSITO"] + Keys.ENTER)

    tipo_frete = navegador.find_element(By.XPATH, '//*[@id="ddlInd_FreightType"]')
    scroll_into_view(tipo_frete)
    opcao_fbp = navegador.find_element(By.XPATH, '//*[@id="ddlInd_FreightType"]/option[3]')
    opcao_fbp.click()

    navegador.find_element(By.XPATH, '//*[@id="modalPanel_modalDialog"]/div/div[2]/table[2]/tbody/tr/td[3]/div/button').click()
    selecao_frota = navegador.find_element(By.XPATH, '//*[@id="modalPanel_modalDialog"]/div/div[2]/table[2]/tbody/tr/td[3]/div/div/div[1]/input')
    selecao_frota.send_keys(linha["ESPÉCIE"] + Keys.ENTER)

    navegador.find_element(By.XPATH, '//*[@id="modalPanel_modalDialog"]/div/div[2]/table[2]/tbody/tr/td[5]/div/button').click()
    selecao_unidade = navegador.find_element(By.XPATH, '//*[@id="modalPanel_modalDialog"]/div/div[2]/table[2]/tbody/tr/td[5]/div/div/div[1]/input')
    selecao_unidade.send_keys(linha["UNIDADE"] + Keys.ENTER)

    navegador.find_element(By.XPATH, '//*[@id="txtVal_FreightValue"]').send_keys(str(linha["Valor"]))

    fbp = navegador.find_element(By.XPATH, '//*[@id="tkfCod_ErpBusinessPartner"]')
    sugestao_encontrada = False
    tentativas = 0

    while not sugestao_encontrada and tentativas < 1:
        fbp.clear()
        fbp.send_keys(linha["Parceiro de Negócio"])
        scroll_into_view(fbp)
        time.sleep(2)
        try:
            WebDriverWait(navegador, 5).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="ui-id-1"]/li')))
            sugestoes = navegador.find_elements(By.XPATH, '//*[@id="ui-id-1"]/li')
            for sugestao in sugestoes:
                if linha["Parceiro de Negócio"] in sugestao.text:
                    scroll_into_view(sugestao)
                    time.sleep(8)
                    sugestao.click()
                    sugestao_encontrada = True
                    break
        except:
            pass
        time.sleep(1)
        tentativas += 1

    if sugestao_encontrada:
        WebDriverWait(navegador, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modalPanelSaveBtn"]')))
        navegador.find_element(By.XPATH, '//*[@id="modalPanelSaveBtn"]').click()
    else:
        navegador.find_element(By.XPATH, '//*[@id="modalPanelCloseBtn"]').click()
    time.sleep(3)
navegador.quit()

In [6]:
print(tabela.columns)

Index(['Parceiro de Negócio', 'ESPÉCIE', 'UNIDADE', 'Valor'], dtype='object')
